## National Center for Atmospheric Research
## 2017 Climate Informatics Hackathon: California Seasonal Rainfall Prediction Starting Kit
Authors: David John Gagne II, Balazs Kegel, Andy Rhines

## Introduction
<img src="Oroville_dam_spillway_2017-02-11.jpg" width="600px">
_Water flowing over the damaged Oroville Dam Spillway. Image source:_ [William Croyle, California Department of Water Resources](https://commons.wikimedia.org/wiki/File:Oroville_dam_spillway_2017-02-11.jpg)

The state of California receives most of its annual rainfall during the winter months when storms fueled by moisture from the tropical Pacific impact the state. This past winter was the wettest on record for northern California, resulting in massive floods and over 1 billion dollars in damage. Storm runoff into Lake Oroville led to extensive releases of water along the Oroville Dam spillway. The main spillway was damaged during the releases but continued to be used due to the high volume of water. The emergency spillway then had to be used, which prompted fears of containment failure and led to large evacuations downstream of the dam. The cost to repair the spillway is estimated at 400 million dollars.

Some of the flooding problems associated with dams in northern California could be managed better with more accurate seasonal and subseasonal forecasts of rainfall. If water managers had a skilled forecast of expected rainfall, then they could change the distribution of water in northern California to be more resilient to large rainfall events. The mitigation process can take weeks to complete, so seasonal forecast leadtimes are needed for effective mitigation. Current operational seasonal precipitation guidance from the NOAA Climate Prediction Center has no skill above climatology for northern California and is not presented in a way that is useful for water managers at the California Department of Water Resources. Current seasonal precipitation forecasting relies primarilly on teleconnection indices, such as ENSO. However, these indices individually are poorly correlated with northern California winter rainfall. For example, the "Godzilla" El Nino of 2015-2016 was expected to bring very heavy rain to California but no extreme rains appeared in the winter of 2015-16. Last years heavy rains happened when ENSO was closer to a neutral state. Other teleconnections should also have some correlation with California rainfall, but finding the most important connections and how they interact is not a task that can be easily done manually.

The goal for the 2017 Climate Informatics Hackathon is to use the November-averaged atmospheric fields to predict the probability of at least 750 mm of rain in northern California between December and February. The observational record for northern California rainfall only goes back to the early 1920s, which would provide a very limited sample size for machine learning or statistical models. Therefore, we are going to use climate model output from simulations run over the last 1000 years. By using climate model output, we hope to sample better the range of possible combinations of weather patterns and rainfall and fit more complex ML and statistical models.

## NCAR Last Millennium Ensemble
The [NCAR Last Millennium Ensemble](http://www.cesm.ucar.edu/projects/community-projects/LME/) is a set of CESM climate model runs starting in 850 AD and run through 2005. The 850 to 1850 period covers preindustrial conditions and 1850-2005 covers the industrial era. This hackathon uses the 12 full forcing ensemble members, which use the same forcings but had their initial air temperature fields perturbed by a small random roundoff of $10^{-14}$C. The ensemble members thus capture the internal variability within the model's climate system. More information about the Last Millennium Ensemble can be found in its [BAMS paper](http://www.cesm.ucar.edu/projects/community-projects/LME/publications/Otto-Bliesner_BAMS-LME_2016-2.pdf).

## The Data
* TS: temperature at the surface (K)
* PSL: mean sea level pressure (Pa) (not Pumpkin Spice Lattes!)
* TMQ: precipitable water (mm)
* U_500: west-east component of the wind at the 500 mb pressure level (m/s)
* V_500: south-north component of the wind at the 500 mb pressure level (m/s)
* Z3_500: geopotential height of the 500 mb pressure level (m)

## Requirements
The Anaconda or Miniconda Python distributions are recommended for downloading the dependencies for this hackathon. Python 3.5 or greater is highly recommended. The following libraries are required. 
* numpy
* scipy
* matplotlib
* xarray
* pandas
* scikit-learn
* cartopy

Once Anaconda or Miniconda is installed on your local machine, execute the following command to create a conda environment with all of the required libraries. 
```bash
cd ~/ramp_kits/california_rainfall/
conda env create -f environment.yml
source activate ramp_ci_2017
```

In [18]:
%matplotlib inline
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from os.path import join
import cartopy.crs as ccrs
from ipywidgets import interact, SelectionSlider
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

## Loading the Data
Each variable is stored in a separate netCDF file. The data loading function is below. If your laptop does not have a large amount of RAM (< 4 GB), we highly recommend loading a smaller subset of variables.

In [26]:
# Edit this list of variables to load in a smaller subset.
data_vars = ["TS", "PSL", "TMQ", "U_500", "V_500"]

def read_data(path, f_prefix, data_vars):
    X_coll = []
    for data_var in data_vars:
        nc_file = join(path, "data", f_prefix + "_{0}.nc".format(data_var))
        print(nc_file)
        ds = xr.open_dataset(nc_file, decode_times=False)
        ds.load()
        X_coll.append(ds[data_var].stack(enstime=("ens", "time")).transpose("enstime", "lat", "lon"))
        ds.close()
    X_ds = xr.merge(X_coll)
    y = pd.read_csv(join(path, "data", f_prefix + "_precip_90.csv"), index_col="Year")
    y_array = np.concatenate([y[c] for c in y.columns])
    return X_ds, y_array

In [11]:
train_X, train_y = read_data("./", "train", data_vars)

./data/train_TS.nc
./data/train_PSL.nc
./data/train_TMQ.nc
./data/train_U_500.nc
./data/train_V_500.nc
./data/train_Z3_500.nc


In [5]:
train_X["TS"].sel(ens=0, time=334.0).shape

(96, 144)

## Viewing the Data
The training data fields are viewable with this interactive widget. Check out the amount of variability among ensemble members and how different fields are linked with each other.

In [8]:
years = np.arange(850, 2005)
def plot_grid(ens, year, var):
    ti = np.where(year == years)[0][0]
    fig = plt.figure(figsize=(10, 5))
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
    ax.coastlines()
    min_val = train_X[var].min()
    max_val = train_X[var].max()
    cont = ax.contourf(train_X["lon"] - 180, train_X["lat"], 
                       train_X[var].sel(ens=ens, time=train_X["time"].values[ti]),
                       np.linspace(min_val, max_val, 20))
    ax.set_title(var + " " + "Year: {0:d} Member {1}".format(year, ens))
    plt.colorbar(cont)
interact(plot_grid, ens=[0, 1, 2, 3], year=SelectionSlider(options=years.tolist()), 
         var=["TS", "PSL", "TMQ", "U_500", "V_500", "Z3_500"])

A Jupyter Widget

<function __main__.plot_grid>

## Spatial Standardized Anomalies
Seasonal signals in spatial data may be hidden by local variability. The polar regions have larger amounts of temperature variability than the tropics, for example. Machine learning and statistical models may focus on local variability instead of spatial variability if the local variability is not controlled for. One way to do this is to normalize the data based on the temporal mean and standard deviation at each grid point. The code for this is shown below.

In [12]:
train_X_anomalies = xr.merge([(train_X[var] - train_X[var].mean(axis=0)) / (train_X[var].std(axis=0)) for var in data_vars])

Wave patterns and dipoles across the globe are now far more obvious in the standardized anomaly data than they were in the original data.

In [17]:
def plot_anomaly(ens, year, var):
    ti = np.where(year == years)[0][0]
    fig = plt.figure(figsize=(10, 5))
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
    ax.coastlines()
    min_val = -5
    max_val = 5
    cont = ax.contourf(train_X_anomalies["lon"] - 180, train_X_anomalies["lat"], 
                       train_X_anomalies[var].sel(ens=ens, time=train_X_anomalies["time"].values[ti]),
                       np.linspace(min_val, max_val, 11), cmap="RdBu_r")
    ax.set_title(var + " " + "Year: {0:d} Member {1}".format(year, ens))
    plt.colorbar(cont)
interact(plot_anomaly, ens=[0, 1, 2, 3], year=SelectionSlider(options=years.tolist()), 
         var=["TS", "PSL", "TMQ", "U_500", "V_500", "Z3_500"])

A Jupyter Widget

<function __main__.plot_anomaly>

## Cross-Validation
For this contest, the cross-validation will take advantage of the relative independence of the different ensemble members. The cross-validation object trains on data from the 3 training ensemble members and tests on the fourth. A separate public testing dataset consists of 4 other ensemble members.

## The Pipeline
The machine learning pipeline for this constest consists of two components: the feature extractor and the classifier. The feature extractor transforms the spatial fields into a flat table that can be processed by scikit-learn-style machine learning and statistical models. The classifier predicts the probability of 750 mm or greater DJF rainfall. 

The starting kit feature extractor spatially normalizes the data and the performs Prinicpal Component Analysis on each variable, keeping the top 20 components. 

In [20]:
class FeatureExtractor():
    def __init__(self):
        self.means = {}
        self.sds = {}
        # Set which model variables you want to use
        self.variables = ["TS", "PSL", "TMQ"]
        self.pca = {}
        self.num_comps = 20

    def fit(self, X_ds, y): 
        # Store the mean and standard deviation for each variable
        for var in self.variables:
            if var not in self.means.keys():
                self.means[var] = X_ds[var].mean(axis=0).values
                self.sds[var] = X_ds[var].std(axis=0).values
                self.sds[var][self.sds[var] == 0] = 1 
            # Normalize the data
            var_norm = (X_ds[var] - self.means[var]) / self.sds[var]
            # Combine the latitude and longitude dimensions into a flat array
            var_flat = var_norm.stack(latlon=("lat", "lon")).values
            var_flat[np.isnan(var_flat)] = 0 
            # Create and fit the PCA
            self.pca[var] = PCA(n_components=self.num_comps)
            self.pca[var].fit(var_flat)


    def transform(self, X_ds):
        X = np.zeros((np.prod(X_ds[self.variables[0]].shape[:1]), 
                      self.num_comps * len(self.variables)), dtype=np.float32)
        c = 0 
        for var in self.variables:
            # Normalize each variable
            var_norm = (X_ds[var] - self.means[var]) / self.sds[var]
            var_flat = var_norm.stack(latlon=("lat", "lon")).values
            var_flat[np.isnan(var_flat)] = 0 
            # Transform the data into princpal components and add to flat 2D array
            X[:, c:c+self.num_comps] = self.pca[var].transform(var_flat)
            c += self.num_comps
        return X

The classifier applies a logistic regression with L1 (LASSO) regularization to the principal component data. L1 regularization results in sparse coeffients, which means that most coefficients will be set to 0. It is a way to perform implicit feature selection with a linear model without having to evaluate a large combination of input variables as in stepwise forward selection. The C parameter controls the strength of the regularization. A smaller value results in fewer non-zero coefficients while a large value leads to more non-zero coefficients. 

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator

class Classifier(BaseEstimator):
    def __init__(self):
        self.clf = LogisticRegression(C=0.01, penalty="l1")

    def fit(self, X, y): 
        self.clf.fit(X, y)

    def predict_proba(self, X): 
        return self.clf.predict_proba(X)

In [25]:
!source activate ramp; ramp_test_submission

Testing California Winter Extreme Rainfall Prediction
Reading train and test files from ./data ...
./data/train_TS.nc
./data/train_PSL.nc
./data/train_TMQ.nc
./data/train_U_500.nc
./data/train_V_500.nc
./data/test_TS.nc
./data/test_PSL.nc
./data/test_TMQ.nc
./data/test_U_500.nc
./data/test_V_500.nc
./data/train_TS.nc
./data/train_PSL.nc
./data/train_TMQ.nc
./data/train_U_500.nc
./data/train_V_500.nc
Reading cv ...
Training ./submissions/starting_kit ...
CV fold 0
	train auc = 0.813
	valid auc = 0.808
	test auc = 0.783
	train brier_score = 0.08
	valid brier_score = 0.093
	test brier_score = 0.077
	train brier_skill_score = 0.158
	valid brier_skill_score = 0.15
	test brier_skill_score = 0.112
/Users/dgagne/anaconda3/envs/ramp/lib/python3.6/site-packages/ramp_workflow-0.1.dev0-py3.6.egg/rampwf/score_types/brier_score.py:99: RuntimeWarning: invalid value encountered in true_divide
  pos_obs_rel_freq = pos_obs_freq / fore_freq
	train brier_score_reliability = 0.026
	valid brier_score_reliab

## Submitting to [ramp.studio](http://ramp.studio)

Once you found a good model, you can submit it to [ramp.studio](http://www.ramp.studio). First, if it is your first time using RAMP, [sign up](http://www.ramp.studio/sign_up), otherwise [log in](http://www.ramp.studio/login). Then find an open event on the particular problem, for example, the event [california_rainfall](https://www.ramp.studio/events/california_rainfall) for this RAMP. Sign up for the event. Both signups are controled by RAMP administrators, so there **can be a delay between asking for signup and being able to submit**.

Once your signup request is accepted, you can go to your [sandbox](http://www.ramp.studio/events/california_rainfall/sandbox) and copy-paste (or upload) [`ts_feature_extractor.py`](/edit/submissions/starting_kit/ts_feature_extractor.py) and [`regressor.py`](/edit/submissions/starting_kit/regressor.py) from `submissions/starting_kit`. Save it, rename it, then submit it. The submission is trained and tested on our backend in the same way as `ramp_test_submission` does it locally. While your submission is waiting in the queue and being trained, you can find it in the "New submissions (pending training)" table in [my submissions](http://www.ramp.studio/events/california_rainfall/my_submissions). Once it is trained, you get a mail, and your submission shows up on the [public leaderboard](http://www.ramp.studio/events/california_rainfall/leaderboard). 
If there is an error (despite having tested your submission locally with `ramp_test_submission`), it will show up in the "Failed submissions" table in [my submissions](http://www.ramp.studio/events/california_rainfall/my_submissions). You can click on the error to see part of the trace.

After submission, do not forget to give credits to the previous submissions you reused or integrated into your submission.

The data set we use at the backend is usually different from what you find in the starting kit, so the score may be different.

The usual way to work with RAMP is to explore solutions, add feature transformations, select models, perhaps do some AutoML/hyperopt, etc., _locally_, and checking them with `ramp_test_submission`. The script prints mean cross-validation scores 
```
----------------------------
train rmse = 0.896 ± 0.0093
valid rmse = 0.799 ± 0.0675
test rmse = 0.97 ± 0.0115
```
The official score in this RAMP (the first score column after "historical contributivity" on the [leaderboard](http://www.ramp.studio/events/california_rainfall/leaderboard)) is root mean squared error ("rmse"), so the line that is relevant in the output of `ramp_test_submission` is `valid rmse = 0.799 ± 0.0675`. When the score is good enough, you can submit it at the RAMP.